# Getting TMDB Data for multi-class classification

In [1]:
#Using API
url="https://api.themoviedb.org/3/movie/top_rated?api_key=7ce14097cd9673cf8aa48a9a52eee5f9&language=en-US&page="

In [2]:
total_pages=471

In [3]:
# Getting data of all the pages
import requests
import pandas as pd

In [4]:
data_list=[]

In [5]:

for i in range(1,total_pages+1):
    data=requests.get(url+str(i))
    data_list.extend(data.json()['results'])
    

In [6]:
df=pd.DataFrame(data_list)

In [7]:
df=df[['title','overview','genre_ids']]

In [8]:
df.sample(5)

,title,overview,genre_ids
48,Howl's Moving Castle,"When Sophie, a shy young woman, is cursed with...","[14, 16, 12]"
4292,Another Earth,On the night of the discovery of a duplicate E...,"[18, 878]"
9014,Final Girl,"Veronica, the new girl in town, is lured into ...","[28, 18, 53]"
5055,It's Complicated,"Ten years after their divorce, Jane and Jake A...","[35, 10749]"
9281,Cell,When a strange signal pulsates through all cel...,"[27, 878, 53]"


In [9]:
genre_data=requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=7ce14097cd9673cf8aa48a9a52eee5f9&language=en-US')

In [10]:
gl=genre_data.json()['genres']

In [11]:
genre_dict={}

In [12]:
#Updating a dictionary of genres
for i in range(len(gl)):
    genre_dict[gl[i]['id']]=gl[i]['name']

In [13]:
#Function to convert genre_ids to genres
def genre_text(glist):
    l=[]
    for i in glist:
       l.append(genre_dict[i])
    return l


In [14]:
df['genre_ids']=df['genre_ids'].apply(genre_text)

In [15]:
df

,title,overview,genre_ids
0,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[Drama, Crime]"
2,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
3,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,[Romance]
4,Gabriel's Inferno Part III,The final part of the film adaption of the ero...,"[Romance, Comedy]"
...,...,...,...
9410,Alone in the Dark,Edward Carnby is a private investigator specia...,"[Action, Fantasy, Horror]"
9411,Battlefield Earth,"In the year 3000, man is no match for the Psyc...","[Action, Adventure, Science Fiction, War]"
9412,House of the Dead,"Set on an island off the coast, a techno rave ...","[Horror, Action, Thriller]"
9413,Dragonball Evolution,The young warrior Son Goku sets out on a quest...,"[Action, Adventure, Fantasy, Science Fiction, ..."


## Text- Preprocessing